In [1]:
# install required system dependencies
!apt-get install -y xvfb x11-utils  
!apt-get install x11-utils > /dev/null 2>&1
!pip install PyOpenGL==3.1.* \
            PyOpenGL-accelerate==3.1.* \
            gym[box2d]==0.17.* 
!pip install pyglet
!pip install ffmpeg
! pip install pyvirtualdisplay
!pip install Image
!pip install gym-maze-trustycoder83

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


If the directory vid exists and has videos left over from previous tries, its better to clean it up before continuing.


In [2]:
!mkdir ./vid
!rm ./vid/*.*

mkdir: cannot create directory ‘./vid’: File exists


In [3]:
import sys
# import pygame
import numpy as np
# import math
# import base64
# import io
# import IPython
import gym
import gym_maze

# from gym.wrappers import Monitor
# from IPython import display
from pyvirtualdisplay import Display
from gym.wrappers.monitoring import video_recorder

d = Display()
d.start()

# Recording filename
video_name = "./vid/Practical_2.mp4"

# Setup the environment for the maze
env = gym.make("maze-sample-10x10-v0")

# Setup the video
vid = None
vid = video_recorder.VideoRecorder(env,video_name)

# env = gym.wrappers.Monitor(env,'./vid',force=True)
#current_state = env.reset()


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import random

def q_learning(discount_factor, epsilon):

  current_state = env.reset()

  states_dic = {} #dictionary to keep the states/coordinates of the Q table
  count = 0
  for i in range(10):
    for j in range(10):
      states_dic[i, j] = count
      count+=1
  
  n_actions = env.action_space.n
  
  # Initialize the Q-table to 0
  q_table = np.zeros((len(states_dic),n_actions))

  # Number of episode we will run
  n_episodes = 100

  # Maximum of iteration per episode
  max_iter_episode = 500

  # Initialize the exploration probability to 1
  exploration_proba = epsilon

  # discount factor
  gamma = discount_factor


  # Learning rate
  lr = 1/n_actions

  total_epochs, total_rewards = 0, 0

  rewards_per_episode = []


  # Iterate over episodes
  for e in range(n_episodes):
      
    # We are not done yet
    done = False
    
    # Sum the rewards that the agent gets from the environment
    total_episode_epochs, total_episode_rewards = 0, 0
    
    for i in range(max_iter_episode):
      
      env.unwrapped.render()
      vid.capture_frame()

      current_coordinate_x = int(current_state[0])
      current_coordinate_y = int(current_state[1])
      current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

      if random.uniform(0, 1) < exploration_proba:
        action = env.action_space.sample() # Explore action space
      else:
        action = int(np.argmax(q_table[current_Q_table_coordinates])) # Exploit learned values

        
      next_state, reward, done, info = env.step(action) 

      next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
      next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary

      
      # Update our Q-table using the Q-learning iteration
      next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]

      # new_value = (1 - learning_rate) * current_value + learning_rate * (reward + gamma * next_max)
      q_table[current_Q_table_coordinates, action] = (1-lr) * q_table[current_Q_table_coordinates, action] +lr*(reward + gamma * max(q_table[next_Q_table_coordinates,:]))

  
      total_episode_rewards = total_episode_rewards + reward
      total_episode_epochs = total_episode_epochs + 1
        
      if done:
        break
      
      current_state = next_state
    
    #Reset enviroment for next episode
    current_state = env.reset()
        
      

    total_epochs = total_epochs + total_episode_epochs
    total_rewards = total_rewards + total_episode_rewards


    #Show the total episode reward        
    #print("Total episode reward:", total_episode_rewards)
    
    rewards_per_episode.append(total_episode_rewards)
    
  print(f"Results after {n_episodes} episodes:")
  print(f"Average timesteps per episode: {total_epochs / n_episodes}")
  print(f"Average rewards per episode: {total_rewards / n_episodes}")

In [5]:
q_learning(discount_factor = 0.5, epsilon = 0.1)
print()

q_learning(discount_factor = 0.5, epsilon = 0.3)
print()

q_learning(discount_factor = 0.5, epsilon = 0.5)
print()

q_learning(discount_factor = 0.5, epsilon = 0.7)

Results after 100 episodes:
Average timesteps per episode: 86.86
Average rewards per episode: 0.8941199999999999

Results after 100 episodes:
Average timesteps per episode: 95.28
Average rewards per episode: 0.8656799999999997

Results after 100 episodes:
Average timesteps per episode: 112.16
Average rewards per episode: 0.8487999999999997

Results after 100 episodes:
Average timesteps per episode: 139.66
Average rewards per episode: 0.8012799999999999


In [6]:
q_learning(discount_factor = 0.1, epsilon = 0.1)
print()

q_learning(discount_factor = 0.3, epsilon = 0.1)
print()

q_learning(discount_factor = 0.5, epsilon = 0.1)
print()

q_learning(discount_factor = 0.7, epsilon = 0.1)

Results after 100 episodes:
Average timesteps per episode: 431.44
Average rewards per episode: -0.0911000000000002

Results after 100 episodes:
Average timesteps per episode: 102.87
Average rewards per episode: 0.8580899999999996

Results after 100 episodes:
Average timesteps per episode: 88.61
Average rewards per episode: 0.88236

Results after 100 episodes:
Average timesteps per episode: 82.56
Average rewards per episode: 0.8884099999999998


**Realistic Initialisation with discount factor = 0.7 and exploration_probability = 0.1**

In [7]:
current_state = env.reset()

states_dic = {} #dictionary to keep the states/coordinates of the Q table
count = 0
for i in range(10):
  for j in range(10):
    states_dic[i, j] = count
    count+=1

n_actions = env.action_space.n

# Initialize the Q-table to 0
q_table = np.zeros((len(states_dic),n_actions))

# Number of episode we will run
n_episodes = 100

# Maximum of iteration per episode
max_iter_episode = 500

# Initialize the exploration probability to 1
exploration_proba = 0.1

# discount factor
gamma = 0.7


# Learning rate
lr = 1/n_actions

total_epochs, total_rewards = 0, 0

rewards_per_episode = []


# Iterate over episodes
for e in range(n_episodes):
    
  # We are not done yet
  done = False
  
  # Sum the rewards that the agent gets from the environment
  total_episode_epochs, total_episode_rewards = 0, 0
  
  for i in range(max_iter_episode):
    
    env.unwrapped.render()
    vid.capture_frame()

    current_coordinate_x = int(current_state[0])
    current_coordinate_y = int(current_state[1])
    current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

    if random.uniform(0, 1) < exploration_proba:
      action = env.action_space.sample() # Explore action space
    else:
      action = int(np.argmax(q_table[current_Q_table_coordinates])) # Exploit learned values

      
    next_state, reward, done, info = env.step(action) 

    next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
    next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary

    
    # Update our Q-table using the Q-learning iteration
    next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]

    # new_value = (1 - learning_rate) * current_value + learning_rate * (reward + gamma * next_max)
    q_table[current_Q_table_coordinates, action] = (1-lr) * q_table[current_Q_table_coordinates, action] +lr*(reward + gamma * max(q_table[next_Q_table_coordinates,:]))


    total_episode_rewards = total_episode_rewards + reward
    total_episode_epochs = total_episode_epochs + 1
      
    if done:
      break
    
    current_state = next_state
  
  #Reset enviroment for next episode
  current_state = env.reset()
      
  total_epochs = total_epochs + total_episode_epochs
  total_rewards = total_rewards + total_episode_rewards

  #Show the total episode reward        
  #print("Total episode reward:", total_episode_rewards)
  
  rewards_per_episode.append(total_episode_rewards)
  
print(f"Results after {n_episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / n_episodes}")
print(f"Average rewards per episode: {total_rewards / n_episodes}")

Results after 100 episodes:
Average timesteps per episode: 81.18
Average rewards per episode: 0.8897899999999994


**Initialising the optimal initial value**

In [12]:
mean_each_actions = q_table.mean(axis=0)
print("Mean value for each action: {}".format(mean_each_actions), '\n')

# initialise the q-value for each action above the mean
initialised_values = mean_each_actions + 0.05
print("Initialised values for each action: {}".format(initialised_values))

Mean value for each action: [0.00608487 0.00856374 0.0078227  0.02849151] 

Initialised values for each action: [0.05608487 0.05856374 0.0578227  0.07849151]


In [14]:
q_table_op = q_table = np.zeros((len(states_dic),n_actions))

q_table_op = q_table_op + initialised_values

#print(q_table_op)


In [15]:
# Number of episode we will run
n_episodes = 100

# Maximum of iteration per episode
max_iter_episode = 500

# Initialize the exploration probability to 1
exploration_proba = 0.1

# discount factor
gamma = 0.7


# Learning rate
lr = 1/n_actions

total_epochs, total_rewards = 0, 0

rewards_per_episode = []


# Iterate over episodes
for e in range(n_episodes):
    
  # We are not done yet
  done = False
  
  # Sum the rewards that the agent gets from the environment
  total_episode_epochs, total_episode_rewards = 0, 0
  
  for i in range(max_iter_episode):
    
    env.unwrapped.render()
    vid.capture_frame()

    current_coordinate_x = int(current_state[0])
    current_coordinate_y = int(current_state[1])
    current_Q_table_coordinates = states_dic[current_coordinate_x, current_coordinate_y]

    if random.uniform(0, 1) < exploration_proba:
      action = env.action_space.sample() # Explore action space
    else:
      action = int(np.argmax(q_table_op[current_Q_table_coordinates])) # Exploit learned values

      
    next_state, reward, done, info = env.step(action) 

    next_coordinate_x = int(next_state[0]) #get coordinates to be used in dictionary
    next_coordinate_y = int(next_state[1]) #get coordinates to be used in dictionary

    
    # Update our Q-table using the Q-learning iteration
    next_Q_table_coordinates = states_dic[next_coordinate_x, next_coordinate_y]

    # new_value = (1 - learning_rate) * current_value + learning_rate * (reward + gamma * next_max)
    q_table_op[current_Q_table_coordinates, action] = (1-lr) * q_table_op[current_Q_table_coordinates, action] +lr*(reward + gamma * max(q_table_op[next_Q_table_coordinates,:]))


    total_episode_rewards = total_episode_rewards + reward
    total_episode_epochs = total_episode_epochs + 1
      
    if done:
      break
    
    current_state = next_state
  
  #Reset enviroment for next episode
  current_state = env.reset()
      
  total_epochs = total_epochs + total_episode_epochs
  total_rewards = total_rewards + total_episode_rewards

  #Show the total episode reward        
  #print("Total episode reward:", total_episode_rewards)
  
  rewards_per_episode.append(total_episode_rewards)
  
print(f"Results after {n_episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / n_episodes}")
print(f"Average rewards per episode: {total_rewards / n_episodes}")

Results after 100 episodes:
Average timesteps per episode: 78.42
Average rewards per episode: 0.9125699999999999
